<img src="http://oproject.org/tiki-download_file.php?fileId=8&display&x=450&y=128" width="200" height="200">
<img src="http://gfif.udea.edu.co/root/tmva/img/tmva_logo.gif" width="200" height="200">

# Autoencoder Variable Transformation
<hr style="border-top-width: 4px; border-top-color: #34609b;"> 

### Initialize DataLoader and Factory 

In [1]:
TMVA::Tools::Instance();
TFile *inputFile = TFile::Open( "../datasets/mydataset.root"); 
TFile* outputFile = TFile::Open( "mydataset_output.root", "RECREATE" );
TMVA::Factory *factory = new TMVA::Factory("TMVARegression", outputFile, 
                                           "!V:!Silent:Color:DrawProgressBar:AnalysisType=Regression" );
TMVA::DataLoader *loader=new TMVA::DataLoader("mydataset");

--- Factory                  : You are running ROOT Version: 6.07/07, Apr 1, 2016
--- Factory                  : 
--- Factory                  : _/_/_/_/_/ _|      _|  _|      _|    _|_|   
--- Factory                  :    _/      _|_|  _|_|  _|      _|  _|    _| 
--- Factory                  :   _/       _|  _|  _|  _|      _|  _|_|_|_| 
--- Factory                  :  _/        _|      _|    _|  _|    _|    _| 
--- Factory                  : _/         _|      _|      _|      _|    _| 
--- Factory                  : 
--- Factory                  : ___________TMVA Version 4.2.1, Feb 5, 2015
--- Factory                  : 


### Setup DataLoader

In [2]:
TMVA::DataLoader *loader2=new TMVA::DataLoader("mydataset");
loader->AddVariable("var0", 'F');
loader->AddVariable("var1", 'F');
loader->AddVariable("var2", 'F');
loader->AddVariable("var3 := var0-var1", 'F');
loader->AddVariable("var4 := var0*var2", 'F');
loader->AddVariable("var5 := var1+var2", 'F');
loader->AddTarget("var6 := 2*var0*var0+4*var1+5*var2", "F");
TTree *regtree = (TTree*)inputFile->Get("MyMCSig");
loader->AddRegressionTree(regtree, 1.0);
loader->PrepareTrainingAndTestTree("", "SplitMode=random:!V");

--- DataSetInfo              : Dataset[mydataset] : Added class "Regression"	 with internal class number 0
--- mydataset                : Add Tree MyMCSig of type Regression with 5449 events
--- DataSetInfo              : Dataset[mydataset] : Class index : 0  name : Regression


### Setup DNN for Autoencoder training

In [ ]:
TString layoutString ("Layout=TANH|3,LINEAR");
TString training0 ("LearningRate=1e-1,Momentum=0.0,Repetitions=1,ConvergenceSteps=300,BatchSize=20,TestRepetitions=15,WeightDecay=0.001,Regularization=NONE,DropConfig=0.0+0.5+0.5+0.5,DropRepetitions=1,Multithreading=True");
TString training1 ("LearningRate=1e-2,Momentum=0.5,Repetitions=1,ConvergenceSteps=300,BatchSize=30,TestRepetitions=7,WeightDecay=0.001,Regularization=L2,Multithreading=True,DropConfig=0.0+0.1+0.1+0.1,DropRepetitions=1");
// TString training2 ("LearningRate=1e-2,Momentum=0.3,Repetitions=1,ConvergenceSteps=300,BatchSize=40,TestRepetitions=7,WeightDecay=0.0001,Regularization=L2,Multithreading=True");
// TString training3 ("LearningRate=1e-3,Momentum=0.1,Repetitions=1,ConvergenceSteps=200,BatchSize=70,TestRepetitions=7,WeightDecay=0.0001,Regularization=NONE,Multithreading=True");

// TString training0 ("LearningRate=1e-5,Momentum=0.5,Repetitions=1,ConvergenceSteps=500,BatchSize=50,TestRepetitions=7,WeightDecay=0.01,Regularization=NONE,DropConfig=0.5+0.5+0.5+0.5,DropRepetitions=2,Multithreading=True");
// TString training1 ("LearningRate=1e-5,Momentum=0.9,Repetitions=1,ConvergenceSteps=170,BatchSize=30,TestRepetitions=7,WeightDecay=0.01,Regularization=L2,DropConfig=0.1+0.1+0.1,DropRepetitions=1,Multithreading=True");
// TString training2 ("LearningRate=1e-5,Momentum=0.3,Repetitions=1,ConvergenceSteps=150,BatchSize=40,TestRepetitions=7,WeightDecay=0.01,Regularization=NONE,Multithreading=True");
// TString training3 ("LearningRate=1e-6,Momentum=0.1,Repetitions=1,ConvergenceSteps=500,BatchSize=100,TestRepetitions=7,WeightDecay=0.0001,Regularization=NONE,Multithreading=True");

TString trainingStrategyString ("TrainingStrategy=");
// trainingStrategyString += training0 + "|" + training1 + "|" + training2 + "|" + training3;
trainingStrategyString += training0 + "|" + training1;
TString nnOptions ("AE(indexLayer=1;pretraining=false;!H:V:ErrorStrategy=SUMOFSQUARES:VarTransform=G:WeightInitialization=XAVIERUNIFORM");
nnOptions.Append (":");
nnOptions.Append (layoutString);
nnOptions.Append (":");
nnOptions.Append (trainingStrategyString);
nnOptions.Append (")");

cout << nnOptions.Data() << endl;

AE(indexLayer=1;pretraining=false;!H:V:ErrorStrategy=SUMOFSQUARES:VarTransform=G:WeightInitialization=XAVIERUNIFORM:Layout=TANH|3,LINEAR:TrainingStrategy=LearningRate=1e-1,Momentum=0.0,Repetitions=1,ConvergenceSteps=300,BatchSize=20,TestRepetitions=15,WeightDecay=0.001,Regularization=NONE,DropConfig=0.0+0.5+0.5+0.5,DropRepetitions=1,Multithreading=True|LearningRate=1e-2,Momentum=0.5,Repetitions=1,ConvergenceSteps=300,BatchSize=30,TestRepetitions=7,WeightDecay=0.001,Regularization=L2,Multithreading=True,DropConfig=0.0+0.1+0.1+0.1,DropRepetitions=1)


### Apply Autoencoder Transform

In [ ]:
TMVA::DataLoader* newloader = loader->VarTransform(nnOptions);

--- mydataset                : [AE Transform] indexLayer = 1
--- mydataset                : [AE Transform] Pretraining = false
--- mydataset                : Preparing DataLoader for Autoencoder Transform DNN Training
--- DataSetFactory           : Dataset[mydataset] : Splitmode is: "RANDOM" the mixmode is: "SAMEASSPLITMODE"
--- DataSetFactory           : Dataset[mydataset] : Create training and testing trees -- looping over class "Regression" ...
--- DataSetFactory           : Dataset[mydataset] : Weight expression for class 'Regression': ""
--- DataSetFactory           : Dataset[mydataset] : Number of events in input trees (after possible flattening of arrays):
--- DataSetFactory           : Dataset[mydataset] :     Regression      -- number of events       : 5449   / sum of weights: 5449 
--- DataSetFactory           : Dataset[mydataset] :     Regression tree -- total number of entries: 5449 
--- DataSetFactory           : Dataset[mydataset] : Preselection: (will NOT affect number o


learning rate reduced to 0.0015625



learning rate reduced to 0.00078125


--- DNN                      : 
--- DNN                      : Training with learning rate = 0.01, momentum = 0.5, repetitions = 1
--- DNN                      : Drop configuration
--- DNN                      :     drop repetitions = 1
--- DNN                      :     Layer 0 = 0
--- DNN                      :     Layer 1 = 0.1
--- DNN                      :     Layer 2 = 0.1
--- DNN                      :     Layer 3 = 0.1
--- DNN                      : 



learning rate reduced to 0.005



learning rate reduced to 
learning rate reduced to 0.0025
0.00125


In [ ]:
// Check number of variables in new Loader
newloader->GetDataSetInfo().GetNVariables();

# Higgs Dataset

In [ ]:
// TMVA::Tools::Instance();
// TString path = "../datasets/higgs-dataset.root";
// if (gSystem->AccessPathName( path ))  // file does not exist in local directory
//         gSystem->Exec("curl -O http://files.oproject.org/root/tmva/datasets/higgs/higgs-dataset.root");
// TFile *input = TFile::Open( path );
// TMVA::DataLoader *loader2=new TMVA::DataLoader("higgs-dataset");
// TTree *Tsignal     = (TTree*)input->Get("TreeS");
// TTree *Tbackground = (TTree*)input->Get("TreeB");
// TString outfileName( "higgs_output.root" );
// TFile* outputFile = TFile::Open( outfileName, "RECREATE" );
// TMVA::Factory *factory = new TMVA::Factory( "TMVAClassification", outputFile,
//             "!V:!Silent:Color:DrawProgressBar:AnalysisType=Classification" );

In [ ]:
// loader2->AddVariable("lepton_pT",'F');
// loader2->AddVariable("lepton_eta",'F');
// loader2->AddVariable("lepton_phi",'F');
// loader2->AddVariable("missing_energy_magnitude",'F');
// loader2->AddVariable("missing_energy_phi",'F');
// loader2->AddVariable("jet_1_pt",'F');
// loader2->AddVariable("jet_1_eta",'F');
// loader2->AddVariable("jet_1_phi",'F');
// loader2->AddVariable("jet_1_b_tag",'F');
// loader2->AddVariable("jet_2_pt",'F');
// loader2->AddVariable("jet_2_eta",'F');
// loader2->AddVariable("jet_2_phi",'F');
// loader2->AddVariable("jet_2_b_tag",'F');
// loader2->AddVariable("jet_3_pt",'F');
// loader2->AddVariable("jet_3_eta",'F');
// loader2->AddVariable("jet_3_phi",'F');
// loader2->AddVariable("jet_3_b_tag",'F');
// loader2->AddVariable("jet_4_pt",'F');
// loader2->AddVariable("jet_4_eta",'F');
// loader2->AddVariable("jet_4_phi",'F');
// loader2->AddVariable("jet_4_b_tag",'F');
// loader2->AddVariable("m_jj",'F');
// loader2->AddVariable("m_jjj",'F');
// loader2->AddVariable("m_lv",'F');
// loader2->AddVariable("m_jlv",'F');
// loader2->AddVariable("m_bb",'F');
// loader2->AddVariable("m_wbb",'F');
// loader2->AddVariable("m_wwbb",'F');

In [ ]:
// Double_t signalWeight     = 1.0;
// Double_t backgroundWeight = 1.0;
// loader2->AddSignalTree    ( Tsignal,     signalWeight     );
// loader2->AddBackgroundTree( Tbackground, backgroundWeight );
// TCut myCuts = ""; 
// TCut myCutb = "";
// loader2->PrepareTrainingAndTestTree( myCuts, myCutb,
//                                          "nTrain_Signal=1829123:nTrain_Background=1170877:nTest_Signal=4000000:nTest_Background=4000000:SplitMode=Random:NormMode=NumEvents:!V" );

In [ ]:
// calculates min, max, mean, RMS and variance of all variables
// loader2->CalcNorm();

In [ ]:
// TString layoutString ("Layout=TANH|100,TANH|50,TANH|10,LINEAR");
// TString training0 ("LearningRate=1e-1,Momentum=0.0,Repetitions=1,ConvergenceSteps=300,BatchSize=20,TestRepetitions=15,WeightDecay=0.001,Regularization=NONE,DropConfig=0.0+0.5+0.5+0.5,DropRepetitions=1,Multithreading=True");
// TString training1 ("LearningRate=1e-2,Momentum=0.5,Repetitions=1,ConvergenceSteps=300,BatchSize=30,TestRepetitions=7,WeightDecay=0.001,Regularization=L2,Multithreading=True,DropConfig=0.0+0.1+0.1+0.1,DropRepetitions=1");
// TString training2 ("LearningRate=1e-2,Momentum=0.3,Repetitions=1,ConvergenceSteps=300,BatchSize=40,TestRepetitions=7,WeightDecay=0.0001,Regularization=L2,Multithreading=True");
// TString training3 ("LearningRate=1e-3,Momentum=0.1,Repetitions=1,ConvergenceSteps=200,BatchSize=70,TestRepetitions=7,WeightDecay=0.0001,Regularization=NONE,Multithreading=True");

// TString trainingStrategyString ("TrainingStrategy=");
// trainingStrategyString += training0 + "|" + training1 + "|" + training2 + "|" + training3;
// // TString nnOptions ("AE(!H:V:ErrorStrategy=CROSSENTROPY:VarTransform=G:WeightInitialization=XAVIERUNIFORM");
// // nnOptions.Append (":");
// // nnOptions.Append (layoutString);
// // nnOptions.Append (":");
// // nnOptions.Append (trainingStrategyString);
// // nnOptions.Append (")");

// TString nnOptions ("!H:V:ErrorStrategy=CROSSENTROPY:VarTransform=G:WeightInitialization=XAVIERUNIFORM");
// nnOptions.Append (":");
// nnOptions.Append (layoutString);
// nnOptions.Append (":");
// nnOptions.Append (trainingStrategyString);

In [ ]:
// print nnOptions string 
// cout << nnOptions.Data() << endl;

In [ ]:
// TMVA::DataLoader* newloader2 = loader2->VarTransform(nnOptions);